In [2]:
import random
import tensorflow as tf
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.12.0


In [4]:
NUM_CLASSES = 11

class Model(tf.keras.Model):
    
    def __init__(self):
        super(Model, self).__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(32, (5, 5), activation='relu', padding='same')
        self.conv2 = tf.keras.layers.Conv2D(64, (5, 5), activation='relu', padding='same')
        self.fc1 = tf.keras.layers.Dense(256, activation='relu')
        self.fc2 = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
        self.max_pool = tf.keras.layers.MaxPool2D((2, 2), (2, 2), padding='same')
        
    def call(self, inp, proposals=None):
        if proposals is None: # Режим обучения
            
            out = self.conv1(inp)
            out = self.max_pool(out)
            out = self.conv2(out)
            out = self.max_pool(out)
            out = tf.keras.layers.Flatten()(out)
            out = self.fc1(out)
            out = self.fc2(out)
            return out
    
        else: # Режим предсказания
            assert inp.shape[0] == 1 # Только batch size = 1
            predictions = []
            roi_pool_size = (7, 7)

            # Извлечение признаков из всей картинки
            out = self.conv(inp)
            out = self.max_pool(out)
            out = self.conv2(out)
            feat = self.max_pool(out)

        # Для каждого пропозала
        for proposal in proposals:
            
            # Отображение координат пространства изображегния
            # в координаты постранства признаков
            ry, rx, rh, rw = proposal
            box_y = int(round(ry * int(feat.shape[1])))
            box_x = int(round(rx * int(feat.shape[2])))
            box_w = int(round(rw * int(feat.shape[2])))
            box_h = int(round(rh * int(feat.shape[1])))
            
            # Вырезаем признаки, относящиеся к пропозалу
            feat_sub = feat[:, box_y: box_y+box_h, box_x: box_x+box_w, :]
            
            # Аналог ROI Pooling
            feat_pooled = tf.image.resize(
                feat_sub,
                (roi_pool_size[0], roi_pool_size[1]),
                tf.image.ResizeMethod.BILINEAR)
            
            # Финальная классификация
            out = tf.keras.layers.Flatten()(feat_pooled)
            out = self.fc1(out)
            out = self.fc2(out)
            
            # Фильтрация класса "фон"
            assert out.shape[0] == 1 # Только batch size = 1
            pred = out[0]
            pred_cls = np.argmax(pred)
            if pred_cls != 10:
                predictions.append([pred_cls] + proposal)
            
        return predictions

model = Model()


Metal device set to: Apple M2


In [5]:
# draw model
tf.keras.utils.plot_model(model, show_shapes=True)

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [6]:
idx = random.randint(0, 1000)
img = test_x_det[idx]
labels_true = test_y_det[idx]
proposals_img = test_proposals[idx]

preds = model(img[None, ...], proposals_img)

NameError: name 'test_x_det' is not defined

In [7]:
show_prediction(img, labels_true)

NameError: name 'show_prediction' is not defined